In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
import IPython

Reading in Data

In [3]:
"""Katies reading in files"""
players = pd.read_csv("players.csv")
week = pd.read_csv("tracking_week_1.csv")
plays = pd.read_csv("plays.csv")
tackles = pd.read_csv("tackles.csv")


# join player positioning information onto a week's worth of tracking data 
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

FileNotFoundError: [Errno 2] No such file or directory: 'players.csv'

In [4]:
""" Matts reading in files """
def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
plays = load_dataset("plays")
players = load_dataset("players")
week = load_dataset("tracking_week_1")
tackles = load_dataset("tackles")
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

(1407439, 18)

Defining Functions for feature engineering at each level

(Just put in ones I thought of, if it has #TO DO: over it then it isnt complete/not started)

In [5]:
"""Game level functions"""

def get_frames_of_catch(games):
    """
    Returns only the frame of the play where the ball is caught.

    Parameters:
    - games: Dataset of games

    Returns:
    - frame: The tracking data of only the data where the ball is caught
    """
    return games.loc[games['event'] == 'pass_outcome_caught'].copy()

In [94]:
"""Frame level functions"""
def distance_to_ball(play_data, frame):
    """
    Calculates the distance to the ball for each player per frame in a play.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    # ball_carrier_id = play_data["ballCarrierId"].iloc[0]
    # ball_carrier = frame[frame["nflId"] == ball_carrier_id]
    frame = frame[frame["club"] != "football"]
    ball_x = frame["x"].values[0]
    ball_y = frame["y"].values[0]
    frame['distance_to_ball_carrier'] = cdist(frame[['x', 'y']], [[ball_x, ball_y]], metric='euclidean')
    return frame


# def create_dnn_input_df(frames, side_of_ball):
#     """
#     Creates the input dataframe for the DNN model.

#     Parameters:
#     - frames: Name of the dataset to load
#     """
#     sorted = frames.sort_values(by='distance_to_ball_carrier').reset_index(drop=True)
    
#     input_df = pd.DataFrame()
#     # Iterate over each row in the sorted DataFrame
#     for idx, row in sorted.iterrows():
#         # Extract player's x and y values
#         x = row['x']
#         y = row['y']
#         s = row['s']
#         a = row['a']
#         dis = row['dis']
#         o = row['o']
#         dir = row['dir']
#         distance_to_ball_carrier = row['distance_to_ball_carrier']


#         # Add x_i columns to the new DataFrame
#         input_df.loc[0,f'{side_of_ball}_x_{idx}'] = x
#         input_df.loc[0,f'{side_of_ball}_y_{idx}'] = y
#         input_df.loc[0,f'{side_of_ball}_s_{idx}'] = s
    #     input_df.loc[0,f'{side_of_ball}_a_{idx}'] = a
    #     input_df.loc[0,f'{side_of_ball}_dis_{idx}'] = dis
    #     input_df.loc[0,f'{side_of_ball}_o_{idx}'] = o
    #     input_df.loc[0,f'{side_of_ball}_dir_{idx}'] = dir
    #     input_df.loc[0,f'{side_of_ball}_distance_to_ball_carrier_{idx}'] = distance_to_ball_carrier

    # # Concatenate the original DataFrame with the new x_i DataFrame
    # return input_df

def create_dnn_input_df(frames, side_of_ball):
    """
    Creates the input dataframe for the DNN model.

    Parameters:
    - frames: Name of the dataset to load
    """
    sorted_frames = frames.sort_values(by=['position', 'jerseyNumber']).reset_index(drop=True)
    
    # Initialize a dictionary to hold the data
    data_dict = {}

    # Iterate over each row in the sorted DataFrame
    for idx, row in sorted_frames.iterrows():
        # Extract player's x and y values
        data_dict[f'{side_of_ball}_x_{idx}'] = row['x']
        data_dict[f'{side_of_ball}_y_{idx}'] = row['y']
        data_dict[f'{side_of_ball}_s_{idx}'] = row['s']
        data_dict[f'{side_of_ball}_a_{idx}'] = row['a']
        data_dict[f'{side_of_ball}_dis_{idx}'] = row['dis']
        data_dict[f'{side_of_ball}_o_{idx}'] = row['o']
        data_dict[f'{side_of_ball}_dir_{idx}'] = row['dir']
        data_dict[f'{side_of_ball}_distance_to_ball_carrier_{idx}'] = row['distance_to_ball_carrier']

    # Convert the dictionary to a DataFrame
    input_df = pd.DataFrame(data_dict, index=[0])

    return input_df


    

def create_dnn_output_df(frames, tackles_data):
    """
    Creates the output dataframe for the DNN model.

    Parameters:
    - frames: Name of the dataset to load
    - tackles_data: Name of the tackles dataset to load
    """
    #print(len(tackles_data))
    total_num_tackles = len(tackles_data)
    tacklers = []
    for idx, row in tackles_data.iterrows():
        tacklers.append(row['nflId'])


    sorted_frames = frames.sort_values(by=['position', 'jerseyNumber']).reset_index(drop=True)
    
    output_df = pd.DataFrame()
    # Iterate over each row in the sorted DataFrame
    for idx, row in sorted_frames.iterrows():
        # Extract player's x and y values
        if row['nflId'] in tacklers:
            output_df.loc[0,f'tackle_{idx}'] = 1/total_num_tackles
        else:
            output_df.loc[0,f'tackle_{idx}'] = 0

    if total_num_tackles == 0:
        output_df.loc[0,f'tackle_11'] = 1
    else:
        output_df.loc[0,f'tackle_11'] = 0
            
    # Concatenate the original DataFrame with the new x_i DataFrame
    return output_df
    





Iterating through each Play

In [101]:
import warnings
warnings.filterwarnings("ignore")
# Process each game, play, and frame
final_input_list = []
counter = 0
counter2 = 0
break_all = False
for gid in tqdm(week['gameId'].unique(), leave=True):
    # if counter == 1:
    #     break
    # counter += 1
    game = week.loc[week['gameId'] == gid].copy()
    game_plays = plays[plays['gameId'] == gid]
    game_tackles = tackles[tackles['gameId'] == gid]

    for pid in tqdm(game['playId'].unique(), leave=False):
        play = game.loc[game['playId'] == pid].copy()
        play_data = game_plays[game_plays['playId'] == pid]
        play_tackles = game_tackles[game_tackles['playId'] == pid] 
        # if break_all:
        #     break
        
        for fid in tqdm(play['frameId'].unique(), leave=False):
            # counter2 += 1
            # if counter2 == 2:
            #     break_all = True
            #     break
            frame = play.loc[play['frameId'] == fid].copy()

            #Adding distance to ball carrier as a feature in tracking data
            frame = distance_to_ball(play_data, frame)
            #print(defense.head())
            #print("Length of ",len(defense))
            offense = frame[frame["club"] == play_data["possessionTeam"].iloc[0]]
            defense = frame[frame["club"] == play_data["defensiveTeam"].iloc[0]]

            offense_input = create_dnn_input_df(offense, "o")
            #print(offense_input)
            defense_input = create_dnn_input_df(defense, "d")
            #print(defense_input)
            output_df = create_dnn_output_df(defense, play_tackles)
            #print(output_df)
            player_tracking_data = pd.concat([offense_input, defense_input, output_df], axis=1)
            #print(player_tracking_data)
            final_input_list.append(player_tracking_data)



            #Adding who tackled the ball carrier
        
final_input_df = pd.concat(final_input_list)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

In [104]:
final_input_df.to_csv(index=False, path_or_buf="final_input_df_all_frames.csv")

In [102]:
final_input_df.head(50)

,o_x_0,o_y_0,o_s_0,o_a_0,o_dis_0,o_o_0,o_dir_0,o_distance_to_ball_carrier_0,o_x_1,o_y_1,...,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11
0,88.25,28.74,0.67,1.97,0.07,244.21,30.29,1.474890,90.22,32.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.26,28.82,0.82,1.82,0.08,255.51,10.32,1.702997,90.46,31.95,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.27,28.92,1.01,1.60,0.10,263.42,0.28,1.931890,90.66,31.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.26,29.04,1.17,1.26,0.12,270.10,354.00,2.173476,90.82,31.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.24,29.17,1.32,0.99,0.13,277.24,349.27,2.418119,90.94,31.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.21,29.31,1.42,0.64,0.14,282.32,347.15,2.675855,91.02,31.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.18,29.45,1.46,0.36,0.15,286.98,345.97,2.893475,91.06,31.19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.14,29.59,1.44,0.38,0.14,290.23,344.05,3.097241,91.03,31.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.08,29.72,1.40,0.60,0.14,292.46,340.66,3.265287,90.95,30.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.02,29.87,1.45,0.64,0.16,294.28,337.90,3.418918,90.84,30.88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
len(final_input_df)

61193

In [56]:
final_input_df.head(20)

,o_x_0,o_y_0,o_s_0,o_a_0,o_dis_0,o_o_0,o_dir_0,o_distance_to_ball_carrier_0,o_x_1,o_y_1,...,tackle_2,tackle_3,tackle_4,tackle_5,tackle_6,tackle_7,tackle_8,tackle_9,tackle_10,tackle_11
0,88.25,28.74,0.67,1.97,0.07,244.21,30.29,12.236217,90.22,32.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.26,28.82,0.82,1.82,0.08,255.51,10.32,11.791014,90.46,31.95,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.27,28.92,1.01,1.60,0.10,263.42,0.28,11.385684,90.66,31.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.26,29.04,1.17,1.26,0.12,270.10,354.00,11.010223,90.82,31.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.24,29.17,1.32,0.99,0.13,277.24,349.27,10.694943,90.94,31.49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.21,29.31,1.42,0.64,0.14,282.32,347.15,10.456003,91.02,31.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.18,29.45,1.46,0.36,0.15,286.98,345.97,10.253907,91.06,31.19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.14,29.59,1.44,0.38,0.14,290.23,344.05,10.096737,91.03,31.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.08,29.72,1.40,0.60,0.14,292.46,340.66,9.953944,90.95,30.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,88.02,29.87,1.45,0.64,0.16,294.28,337.90,9.817586,90.84,30.88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
len(final_input_df)

1473